# Protein Graph Analytics

Graphein provides built in utilities for computing various analytics relating graph structure and physicochemical properties. These include functions for getting summaries and metrics out as well as interactive plotting functions.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/a-r-j/graphein/blob/master/notebooks/protein_graph_analytics.ipynb)

In [ ]:
# Install graphein if necessary:
# !pip install graphein
# Install DSSP if necessary:
# !sudo apt-get install dssp (better for colab) OR !conda install -c salilab dssp

## Constructing a graph for analysis
First, we need a graph to work with. Here we construct a graph of [3EIY](https://www.rcsb.org/structure/3eiy), a pyrophosphatase. We use the following edges:
* Hydrophobic Interactions
* Aromatic Interactions
* Disulfide Interactions
* Peptide Bonds

Using DSSP, we add relative (RSA) and absolute solvent accessibility (ASA) to the nodes.

In [ ]:
from graphein.protein.config import ProteinGraphConfig, DSSPConfig
from graphein.protein.graphs import construct_graph
from graphein.protein.edges.distance import (
    add_aromatic_interactions,
    add_disulfide_interactions,
    add_hydrophobic_interactions,
    add_peptide_bonds,
)
from graphein.protein.features.nodes import asa, rsa

config = ProteinGraphConfig(
    edge_construction_functions=[       # List of functions to call to construct edges.
        add_hydrophobic_interactions,
        add_aromatic_interactions,
        add_disulfide_interactions,
        add_peptide_bonds,
    ],
    graph_metadata_functions=[asa, rsa],  # Add ASA and RSA features.
    dssp_config=DSSPConfig(),             # Add DSSP config in order to compute ASA and RSA.
)

g = construct_graph(pdb_code="3eiy", config=config)

In [ ]:
from graphein.protein.visualisation import plotly_protein_structure_graph

plotly_protein_structure_graph(g, node_size_multiplier=1)

## Residue Composition
We can plot the residue composition of the graph with `plot_residue_composition`. This can be as a bar (`plot_type="bar"`) or pie chart (`plot_type="pie"`)

In [ ]:
from graphein.protein.analysis import plot_residue_composition

fig = plot_residue_composition(g, sort_by="count", plot_type="pie") # Can also sort by "alphabetical"
fig.show()

In [ ]:
fig = plot_residue_composition(g, sort_by="count", plot_type="bar")
fig.show()

## Graph Summary Statistics
We can compute graph-theoretic summary statistics for a given protein graph based on various centrality measures.

In [ ]:
from graphein.protein.analysis import graph_summary

graph_summary(g)

In [ ]:
graph_summary(g, plot=True)

## Degree Distribution

We can plot the distribution of node degrees over the protein graph

In [ ]:
from graphein.protein.analysis import plot_degree_distribution

fig = plot_degree_distribution(g)
fig.show()

And the total degree broken down by residue type.

In [ ]:
from graphein.protein.analysis import plot_degree_by_residue_type

fig = plot_degree_by_residue_type(g, normalise_by_residue_occurrence=False)
fig.show()

Or the total degree normalised by the occurence of each amino acid type

In [ ]:
fig = plot_degree_by_residue_type(g, normalise_by_residue_occurrence=True)
fig.show()

## Edge Type Distribution

We can plot the distribution of edge types in the graph with `plot_edge_type_distribution()`

In [ ]:
from graphein.protein.analysis import plot_edge_type_distribution

fig = plot_edge_type_distribution(g, plot_type="bar")
fig.show()

In [ ]:
from graphein.protein.analysis import plot_edge_type_distribution

fig = plot_edge_type_distribution(g, plot_type="pie")
fig.show()

## Scatter Matrix

We can use scatter matrices to correlate node-level graph metrics (e.g. centrality measures) with node features.

First, we will plot the scatter matrix for a selection of centrality measures and the relative solvent accessibility (RSA) and absolute solvent accessibility (ASA). *N.B.* we can include these features in the plot as we have added them as metadata in the initial graph construction. Refer to cell 2 at the top of this notebook for clarity.


A summary of the function parameters:

```python

plot_graph_metric_property_correlation(
    g: nx.Graph,                                # Graph to plot
    summary_statistics: List[str] = [           # Graph theoretic metrics to include
        "degree",
        "betweenness_centrality",
        "closeness_centrality",
        "eigenvector_centrality",
        "communicability_betweenness_centrality",
    ],
    properties: List[str] = ["asa", "rsa"],     # Node features to include
    colour_by: Optional[str] = "residue_type",  # How to colour the points
    opacity: float = 0.2,                       # Opacity of markers
    diagonal_visible: bool = True,              # Whether or not to show the leading diagonal of the plot
    title: Optional[str] = None,                # Plot title
    height: int = 1000,                         # Plot height
    width: int = 1000,                          # Plot width
    font_size: int = 10,                        # Font size for axes, title and ticks
)
```

In [ ]:
from graphein.protein.analysis import plot_graph_metric_property_correlation

plot_graph_metric_property_correlation(g, diagonal_visible=False)

In [ ]:
plot_graph_metric_property_correlation(g, diagonal_visible=False, colour_by=None)

However, we can include more features. For instance, we can correlate these metrics with the properties defined by the ExPaSy amino acid scales. We'll plot a subset for the sake of space.

The full list of properties are:

```python
['b_factor', 'pka_cooh_alpha', 'pka_nh3', 'pka_rgroup', 'isoelectric_points', 'molecularweight', 'numbercodons', 'bulkiness', 'polarityzimmerman', 'polaritygrantham', 'refractivity', 'recognitionfactors', 'hphob_eisenberg', 'hphob_sweet', 'hphob_woods', 'hphob_doolittle', 'hphob_manavalan', 'hphob_leo', 'hphob_black', 'hphob_breese', 'hphob_fauchere', 'hphob_guy', 'hphob_janin', 'hphob_miyazawa', 'hphob_argos', 'hphob_roseman', 'hphob_tanford', 'hphob_wolfenden', 'hphob_welling', 'hphob_wilson', 'hphob_parker', 'hphob_ph3_4', 'hphob_ph7_5', 'hphob_mobility', 'hplchfba', 'hplctfa', 'transmembranetendency', 'hplc2_1', 'hplc7_4', 'buriedresidues', 'accessibleresidues', 'hphob_chothia', 'hphob_rose', 'ratioside', 'averageburied', 'averageflexibility', 'alpha_helixfasman', 'beta_sheetfasman', 'beta_turnfasman', 'alpha_helixroux', 'beta_sheetroux', 'beta_turnroux', 'coilroux', 'alpha_helixlevitt', 'beta_sheetlevitt', 'beta_turnlevitt', 'totalbeta_strand', 'antiparallelbeta_strand', 'parallelbeta_strand', 'a_a_composition', 'a_a_swiss_prot', 'relativemutability']
```

These can be accessed programmatically with:
```python
for _, d in g.nodes(data=True):
    print(d.keys())
    break
```

**NB** These are just examples, *any* numerical node feature can be provided - including user-defined features.

In [ ]:
from functools import partial
from graphein.protein.features.nodes import expasy_protein_scale

# Construct the graph with the expasy features.
config = ProteinGraphConfig(
    edge_construction_functions=[
        add_hydrophobic_interactions,
        add_aromatic_interactions,
        add_disulfide_interactions,
        add_peptide_bonds,
    ],
    node_metadata_functions=[partial(expasy_protein_scale, add_separate=True)], # Add expasy scale (add partial it so each feature is added under a separate key)
)
g = construct_graph(
    pdb_code="3eiy",
    config=config
    )

# Plot
plot_graph_metric_property_correlation(
    g,
    diagonal_visible=False,
    colour_by="residue_type",
    properties=[
        "pka_rgroup",
        "isoelectric_points",
        "bulkiness",
        "transmembranetendency",
        "coilroux",
        "relativemutability"
        ]
    )